In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import traceback

In [2]:
mitig = pd.read_csv("mitigation_cleaned.csv")
country = pd.read_csv("Google_US_only_cleaned.csv")
states_only = pd.read_csv("Google_US_and_states_cleaned.csv")

In [3]:
# convert date columns in each data set to be datetime objects
country.loc[:,'date'] = pd.to_datetime(country['date'])
states_only.loc[:,'date'] = pd.to_datetime(states_only['date'])
mitig.loc[:,'Date'] = pd.to_datetime(mitig['Date'])


In [4]:
mitig = mitig[mitig["County"].isna()] # remove entries related to counties

In [49]:
# Find the minimum mobility value in a window after a mitigation event - US only

window_length = 5
window = np.timedelta64(window_length,'D')
google_mob_categories = country.columns[-6:] # mobility data categories


first_date = country["date"].iloc[0]
last_date = country["date"].iloc[-1]

lowest_mobility_val = []
    
# select rows in USA after earliest date  and before last date in google data from mitigation data
us_only = mitig.loc[(mitig["State"] == "USA")]
us_only = us_only.loc[(us_only["Date"] > first_date) & (us_only["Date"] < (last_date - window))]

for i,date in enumerate(us_only["Date"]):
    new_row = {"state": "USA"}
    new_row.update({"mitigation_date": date})
    new_row.update({"mitigation_type": us_only["Mitigation type"].iloc[i]})

    # get min value from each google mob category
    for cat in google_mob_categories:
        short_cat_name = cat[:-29]
        
        # get row index of min value in category for country between mitigation date and date + window
        where = country.loc[(country["date"] >= date) & (country["date"] < date + window)][cat].idxmin()
        # add the drop value
        idx = country.index[country["date"] == date]
        mob_at_event = country[cat].iloc[idx].values[0] # mobility for category at the date of mitigation event
        val = country[cat].iloc[where] - mob_at_event
#         print(f"{country[cat].iloc[where]} - {mob_at_event} = {val}")
        new_row.update({"drop_" + short_cat_name: val})
        # add the date of the value
        min_date = country["date"].iloc[where]
        new_row.update({"date_of_min_" + short_cat_name: min_date})
        #a add the delay
        delay = min_date - date
        new_row.update({"delay_" + short_cat_name: delay.days})
    lowest_mobility_val.append(new_row);
    
lowest_mobility_vals_US = DataFrame(lowest_mobility_val)
                                                   

In [50]:
lowest_mobility_vals_US

,state,mitigation_date,mitigation_type,drop_retail_and_recreation,date_of_min_retail_and_recreation,delay_retail_and_recreation,drop_grocery_and_pharmacy,date_of_min_grocery_and_pharmacy,delay_grocery_and_pharmacy,drop_parks,...,delay_parks,drop_transit_stations,date_of_min_transit_stations,delay_transit_stations,drop_workplaces,date_of_min_workplaces,delay_workplaces,drop_residential,date_of_min_residential,delay_residential
0,USA,2020-02-28,other,-1.0,2020-03-02,3,0.0,2020-02-28,0,0.0,...,0,-3.0,2020-03-02,3,0.0,2020-02-28,0,-1.0,2020-02-29,1
1,USA,2020-02-29,movement restriction,-5.0,2020-03-02,2,-1.0,2020-03-02,2,-11.0,...,2,-5.0,2020-03-02,2,-2.0,2020-03-03,3,0.0,2020-02-29,0
2,USA,2020-03-03,other,-6.0,2020-03-06,3,-7.0,2020-03-06,3,-8.0,...,3,-1.0,2020-03-04,1,0.0,2020-03-03,0,0.0,2020-03-03,0
3,USA,2020-03-06,other,0.0,2020-03-06,0,0.0,2020-03-06,0,0.0,...,0,-5.0,2020-03-10,4,-4.0,2020-03-10,4,0.0,2020-03-06,0
4,USA,2020-03-11,movement restriction,-17.0,2020-03-15,4,0.0,2020-03-11,0,-23.0,...,3,-11.0,2020-03-15,4,-6.0,2020-03-13,2,0.0,2020-03-11,0
5,USA,2020-03-11,movement restriction,-17.0,2020-03-15,4,0.0,2020-03-11,0,-23.0,...,3,-11.0,2020-03-15,4,-6.0,2020-03-13,2,0.0,2020-03-11,0
6,USA,2020-03-13,movement restriction,-20.0,2020-03-17,4,-14.0,2020-03-15,2,-5.0,...,3,-16.0,2020-03-17,4,-20.0,2020-03-17,4,0.0,2020-03-13,0
7,USA,2020-03-13,state of emergency,-20.0,2020-03-17,4,-14.0,2020-03-15,2,-5.0,...,3,-16.0,2020-03-17,4,-20.0,2020-03-17,4,0.0,2020-03-13,0
8,USA,2020-03-15,movement restriction,-19.0,2020-03-19,4,-5.0,2020-03-19,4,-6.0,...,1,-23.0,2020-03-19,4,-26.0,2020-03-19,4,0.0,2020-03-15,0
9,USA,2020-03-15,events cancelled,-19.0,2020-03-19,4,-5.0,2020-03-19,4,-6.0,...,1,-23.0,2020-03-19,4,-26.0,2020-03-19,4,0.0,2020-03-15,0


In [48]:
# Find the minimum mobility value in a window after a mitigation event - States only
window_length = 3
window = np.timedelta64(window_length,'D')
google_mob_categories = states_only.columns[-6:] # mobility data categories


first_date = states_only["date"].unique()[0]
last_date = states_only["date"].unique()[-1]


lowest_mobility_val = []


# Lookup states from google data, remove district of columbia
states = states_only["sub_region_1"].unique()
states = np.delete(states, np.where(states == "District of Columbia"))

for state in states:
    # select rows in region after earliest date  and before last date in google data from mitigation data
    mitig_cur_state = mitig.loc[(mitig["State"] == state)]
    mitig_cur_state = mitig_cur_state.loc[(mitig_cur_state["Date"] >= first_date) & (mitig_cur_state["Date"] < (last_date - window))]
    
    mob_cur_state = states_only.loc[states_only["sub_region_1"] == state]
    mob_cur_state = mob_cur_state.loc[(mob_cur_state["date"] >= first_date) & (mob_cur_state["date"] < (last_date - window))]
    try:
        for i,date in enumerate(mitig_cur_state["Date"]):
            new_row = {"state": state}
            new_row.update({"mitigation_date": date})
            new_row.update({"mitigation_type": mitig_cur_state["Mitigation type"].iloc[i]})

            # get min value from each google mob category
            for cat in google_mob_categories:
                
                short_cat_name = cat[:-29]

                # get row index of min value in category for states_only between mitigation date and date + window
                where = mob_cur_state.loc[(mob_cur_state["date"] >= date) & (mob_cur_state["date"] < date + window)][cat].idxmin()
                # add the drop value
                idx = mob_cur_state.index[mob_cur_state["date"] == date] # index of current date in current state
                mob_at_event = states_only[cat].iloc[idx].values[0] # mobility for category at the date of mitigation event
                val = states_only[cat].iloc[where] - mob_at_event
                new_row.update({"drop_" + short_cat_name: val})

                # add the date
                min_date = states_only["date"].iloc[where]
                new_row.update({"date_of_min_" + short_cat_name: min_date})
                # add the delay
                delay = min_date - date
                
                new_row.update({"delay_" + short_cat_name: delay.days})
            lowest_mobility_val.append(new_row);
    except IndexError:
        print(f"Error: {state}: {date}: {where}")
        traceback.print_exc()
    except ValueError:
        print(f"Error: {state}: {date}: {where}")
        traceback.print_exc()

lowest_mobility_vals_states = DataFrame(lowest_mobility_val)

In [51]:
lowest_mobility_vals_states

,state,mitigation_date,mitigation_type,drop_retail_and_recreation,date_of_min_retail_and_recreation,delay_retail_and_recreation,drop_grocery_and_pharmacy,date_of_min_grocery_and_pharmacy,delay_grocery_and_pharmacy,drop_parks,...,delay_parks,drop_transit_stations,date_of_min_transit_stations,delay_transit_stations,drop_workplaces,date_of_min_workplaces,delay_workplaces,drop_residential,date_of_min_residential,delay_residential
0,Alabama,2020-03-06,new public services,0.0,2020-03-06,0,0.0,2020-03-06,0,0.0,...,0,0.0,2020-03-06,0,0.0,2020-03-06,0,-1.0,2020-03-07,1
1,Alabama,2020-03-11,other,-3.0,2020-03-13,2,0.0,2020-03-11,0,-1.0,...,2,0.0,2020-03-11,0,-3.0,2020-03-13,2,0.0,2020-03-11,0
2,Alabama,2020-03-12,other,-9.0,2020-03-14,2,0.0,2020-03-12,0,-17.0,...,1,-1.0,2020-03-13,1,-3.0,2020-03-13,1,0.0,2020-03-12,0
3,Alabama,2020-03-12,events cancelled,-9.0,2020-03-14,2,0.0,2020-03-12,0,-17.0,...,1,-1.0,2020-03-13,1,-3.0,2020-03-13,1,0.0,2020-03-12,0
4,Alabama,2020-03-12,school closing,-9.0,2020-03-14,2,0.0,2020-03-12,0,-17.0,...,1,-1.0,2020-03-13,1,-3.0,2020-03-13,1,0.0,2020-03-12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,Wyoming,2020-03-27,new public services,-2.0,2020-03-28,1,-5.0,2020-03-29,2,-5.0,...,1,-1.0,2020-03-29,2,0.0,2020-03-27,0,-9.0,2020-03-29,2
1661,Wyoming,2020-03-27,other,-2.0,2020-03-28,1,-5.0,2020-03-29,2,-5.0,...,1,-1.0,2020-03-29,2,0.0,2020-03-27,0,-9.0,2020-03-29,2
1662,Wyoming,2020-04-03,movement restriction,-5.0,2020-04-04,1,-8.0,2020-04-05,2,0.0,...,0,-10.0,2020-04-05,2,0.0,2020-04-03,0,-8.0,2020-04-05,2
1663,Wyoming,2020-04-03,other,-5.0,2020-04-04,1,-8.0,2020-04-05,2,0.0,...,0,-10.0,2020-04-05,2,0.0,2020-04-03,0,-8.0,2020-04-05,2


In [55]:
# mean mobility drops by mitigation type in each category
cols = ['drop_retail_and_recreation', 'drop_grocery_and_pharmacy', 'drop_parks', 'drop_transit_stations', 'drop_workplaces', 'drop_residential']
table = pd.pivot_table(lowest_mobility_vals_states, values=cols, index=['mitigation_type'], aggfunc=np.mean)
table

,drop_grocery_and_pharmacy,drop_parks,drop_residential,drop_retail_and_recreation,drop_transit_stations,drop_workplaces
mitigation_type,,,,,,
events cancelled,-7.432203,-22.745763,-0.516949,-10.355932,-6.423729,-5.601695
movement restriction,-5.800000,-18.088889,-0.711111,-7.525926,-5.162963,-5.348148
new public services,-6.604240,-20.250883,-1.862191,-6.346290,-5.427562,-4.505300
non-essential closing,-9.108696,-17.548913,-1.271739,-10.070652,-7.244565,-7.038043
other,-6.409382,-20.089552,-2.093817,-6.373134,-4.906183,-4.882729
public service reduction,-7.511905,-18.261905,-1.142857,-9.023810,-6.702381,-5.952381
school closing,-6.436364,-23.854545,-0.963636,-8.818182,-6.248485,-5.812121
state of emergency,-5.637681,-23.478261,-0.724638,-7.376812,-5.753623,-4.449275
stay at home,-8.783505,-22.113402,-1.340206,-7.628866,-4.701031,-4.649485


In [56]:
# mean delay to mobility drop by mitigation type in each category
cols = ['delay_retail_and_recreation', 'delay_grocery_and_pharmacy', 'delay_parks', 'delay_transit_stations', 'delay_workplaces', 'delay_residential']
table = pd.pivot_table(lowest_mobility_vals_states, index=['mitigation_type'], values=cols, aggfunc='mean')
table

,delay_grocery_and_pharmacy,delay_parks,delay_residential,delay_retail_and_recreation,delay_transit_stations,delay_workplaces
mitigation_type,,,,,,
events cancelled,1.194915,1.161017,0.228814,1.686441,1.584746,1.423729
movement restriction,1.162963,0.948148,0.318519,1.385185,1.348148,1.362963
new public services,1.095406,0.978799,0.618375,1.265018,1.261484,1.060071
non-essential closing,1.375000,1.038043,0.472826,1.505435,1.516304,1.440217
other,1.187633,0.897655,0.684435,1.292111,1.257996,1.183369
public service reduction,1.309524,1.083333,0.452381,1.547619,1.511905,1.547619
school closing,1.121212,1.103030,0.381818,1.581818,1.290909,1.400000
state of emergency,0.956522,1.159420,0.376812,1.304348,1.362319,1.246377
stay at home,1.237113,0.969072,0.567010,1.309278,1.103093,1.195876


In [48]:
print("US largest drops")
for col in lowest_mobility_vals_US.columns[3::3]:
    print(lowest_mobility_vals_US[col].min())
for i, state in enumerate(lowest_mobility_vals_states["state"].unique()):
    print(f"{i}) {state} largest drops")
    for col in lowest_mobility_vals_states.columns[3::3]:
        where = lowest_mobility_vals_states.loc[lowest_mobility_vals_states['state'] == state][col].idxmin()
        val = lowest_mobility_vals_states[col].iloc[where]
        date = lowest_mobility_vals_states['mitigation_date'].iloc[where]
        mitig_type = lowest_mobility_vals_states['mitigation_type'].iloc[where]
        print("\t{} {}".format(col, val))
        print("\t{} {}".format(date, mitig_type))

US largest drops
-16.0
-18.0
-28.0
-13.0
-20.0
-9.0
0) Alabama largest drops
	drop_retail_and_recreation -27.0
	2020-03-18 00:00:00 events cancelled
	drop_grocery_and_pharmacy -30.0
	2020-04-03 00:00:00 stay at home
	drop_parks -53.0
	2020-03-27 00:00:00 non-essential closing
	drop_transit_stations -19.0
	2020-03-18 00:00:00 events cancelled
	drop_workplaces -26.0
	2020-03-14 00:00:00 public service reduction
	drop_residential -7.0
	2020-03-27 00:00:00 non-essential closing
1) Alaska largest drops
	drop_retail_and_recreation -31.0
	2020-03-15 00:00:00 events cancelled
	drop_grocery_and_pharmacy -17.0
	2020-03-16 00:00:00 public service reduction
	drop_parks -59.0
	2020-03-22 00:00:00 movement restriction
	drop_transit_stations -28.0
	2020-03-15 00:00:00 events cancelled
	drop_workplaces -24.0
	2020-03-15 00:00:00 events cancelled
	drop_residential -11.0
	2020-04-03 00:00:00 wear masks
2) Arizona largest drops
	drop_retail_and_recreation -21.0
	2020-03-15 00:00:00 school closing
	drop_g

	drop_parks -66.0
	2020-03-11 00:00:00 events cancelled
	drop_transit_stations -18.0
	2020-03-18 00:00:00 non-essential closing
	drop_workplaces -26.0
	2020-03-14 00:00:00 public service reduction
	drop_residential -9.0
	2020-04-03 00:00:00 other
42) Utah largest drops
	drop_retail_and_recreation -26.0
	2020-03-16 00:00:00 school closing
	drop_grocery_and_pharmacy -24.0
	2020-03-12 00:00:00 movement restriction
	drop_parks -87.0
	2020-03-17 00:00:00 movement restriction
	drop_transit_stations -18.0
	2020-03-16 00:00:00 school closing
	drop_workplaces -23.0
	2020-03-21 00:00:00 other
	drop_residential -10.0
	2020-03-18 00:00:00 new public services
43) Vermont largest drops
	drop_retail_and_recreation -36.0
	2020-03-16 00:00:00 movement restriction
	drop_grocery_and_pharmacy -31.0
	2020-03-25 00:00:00 public service reduction
	drop_parks -68.0
	2020-03-09 00:00:00 public service reduction
	drop_transit_stations -34.0
	2020-03-11 00:00:00 events cancelled
	drop_workplaces -28.0
	2020-03-1